<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href=#modules, style="font-size: xx-large">1. モジュールインポート</a>
            <ul>※サードパーティライブラリ>>>自作モジュール>>>（ここまで本ipynb外）>>>自作関数（本ipynb内）</ul>
        </summary>
    </details>

<details>
    <summary>
        <a href=#data , style="font-size: xx-large">2. オリジナルデータインポート</a>
    </summary>
</details>

<details>
    <summary>
        <a href="#all", style="font-size: xx-large">3. 全体</a>
    </summary>
    <table></table>
</details>
    
<details>
    <summary>
        <a href="#sepyear", style="font-size: xx-large">4. 期間ごと</a>
    </summary>
</details>
</b>


---


<a id=modules></a>

## **1. モジュールインポート**


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ptick
import networkx as nx
import networkx.algorithms.bipartite as bip

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.6f}'.format


In [ ]:
# 自作モジュールインポート
from Scripts.kci import df_to_kci as dtk
from Scripts.vizualize import rank as vr
from Scripts.vizualize import distribution as vd


In [ ]:
def ccdf(diversity_col: list):
        freq_array = np.array(np.bincount(diversity_col))
        p_list = []
        cumsum = 0.0
        s = float(freq_array.sum())
        for freq in freq_array:
            if freq != 0:
                cumsum += freq / s
                p_list.append(cumsum)
            else:
                p_list.append(1.0)
                
        ccdf_array = 1 - np.array(p_list)
        if ccdf_array[0] == 0:
            ccdf_array[0] = 1.0
        return ccdf_array

color_list = ['red']+[
    'red', 'green', 'blue', 'yellow', 'orange', 'purple', 'pink', 'brown',
    'grey', 'violet', 'indigo', 'turquoise', 'gold', 'lime', 'coral',
    'navy', 'skyblue', 'tomato', 'olive', 'cyan', 'darkred', 'darkgreen',
    'darkblue', 'darkorange', 'darkviolet', 'deeppink', 'firebrick', 'darkcyan',
    'darkturquoise', 'darkslategray', 'darkgoldenrod', 'mediumblue', 'mediumseagreen',
    'mediumpurple', 'mediumvioletred', 'midnightblue', 'saddlebrown', 'seagreen',
    'sienna', 'steelblue'
    ][10:]


In [ ]:
def ccdf(target_array:np.array):
    freq_array = np.array(np.bincount(target_array))
    p_list = []
    cumsum = 0.0
    s = float(freq_array.sum())
    for freq in freq_array:
        if freq != 0:
            cumsum +=freq / s
            p_list.apend(cumsum)
        else:
            p_list.append(1.0)
    ccdf_array = 1 - np.arrau(p_list)
    if ccdf_array[0] == 0: ccdf_array[0] = 1.0
    return ccdf_array


In [ ]:

def descript_network(graph_df:pd.DataFrame, 
                     hr_col:str='right_person_name', 
                     class_col:str='ipc_class', 
                     segment_col:str='1981-2010'
                     ):
    # ネットワークの特性を調べる
    BG = nx.Graph()
    BG.add_nodes_from(graph_df[hr_col].unique(), bipartite=0)
    BG.add_nodes_from(graph_df[class_col].unique(), bipartite=1)
    BG.add_edges_from(list(zip(graph_df[hr_col], graph_df[class_col])))
    
    if nx.is_connected(BG): print('連結性ある')
    else: print('連結性ない')
    
    # 特許権者ノード
    hr_nodes = {n for n, d in BG.nodes(data=True) if d['bipartite'] == 0}
    hr_degree_array = ccdf(dict(bip.degrees(BG, hr_nodes)[1]).values())
    
    
    
    # 特許分類ノード
    ipc_nodes = set(BG) - hr_nodes
    ipc_degree_array = ccdf(dict(bip.degrees(BG, ipc_nodes)[1]).values())
    
    # G = nx.from_pandas_edgelist(graph_df, 'source', 'target', edge_attr=True)
    # print(nx.info(G))
    # print('平均最短距離:', nx.average_shortest_path_length(G))
    # print('平均クラスタリング係数:', nx.average_clustering(G))
    # print('直径:', nx.diameter(G))
    # print('半径:', nx.radius(G))
    # print('次数中心性:', nx.degree_centrality(G))
    # print('媒介中心性:', nx.betweenness_centrality(G))
    # print('固有ベクトル中心性:', nx.eigenvector_centrality(G))
    # print('PageRank:', nx.pagerank(G))
    # print('次数分布:', nx.degree_histogram(G))
    res_dict = {
                'network':BG}
    
    return BG


---


In [ ]:
year_start = 1981
year_end = 2010
year_range = 10

ar = 'app'


In [ ]:
graph_df = pd.read_csv(f'../Data/0_Graph/{ar}_{year_start}_{year_end}.csv', 
                       encoding='utf-8', 
                       sep=',', 
                       dtype={'segment':str, 
                              'right_person_name':str, 
                              'ipc_class':str, 
                              'mcp':np.int64}
                       )
graph_dict = {}
for s in graph_df['segment'].unique():
       
       graph_df = graph_df[graph_df['segment'] == '1981-2010']
# graph_df.groupby('right_person_name')[['mcp']].sum().sort_values('mcp', ascending=False).head(10)


In [ ]:
# BG = descript_network(graph_df)

graph_dict = {}
for s in graph_df['segment'].unique():
    BG = descript_network(graph_df[graph_df['segment'] == s])
    

<a href=#top>先頭に戻る</a>

---


<a id=#description></a>

## **1. 記述統計**

In [ ]:
# この二部グラフって連結性あるん
if nx.is_connected(BG):
    print('ある')
else:
    print('ない')


In [ ]:
# 特許権者ノード
hr_nodes = {n for n, d in BG.nodes(data=True) if d['bipartite'] == 0}

# 特許分類ノード
ipc_nodes = set(BG) - hr_nodes


In [ ]:
# 特許権者ノードの密度
print(round(bip.density(BG, hr_nodes), 3))

# 特許分類ノードの密度
print(round(bip.density(BG, ipc_nodes), 3))


In [ ]:
dict(bip.degrees(BG, hr_nodes)[1]).values()

In [ ]:
color_count = 0
fig, ax = plt.subplots(figsize=(8, 8))
for s in list(right_person_df['segment'].unique())[0:1]:
    ccdf_array = ccdf(right_person_df[right_person_df['segment']==s]['diversity'].to_list())
    ax.plot(range(1, len(ccdf_array)+1), ccdf_array, 'o', markersize=7, 
                    color=color_list[color_count], label=s+'年度', alpha=0.6)
    color_count += 1
    print(right_person_df[right_person_df['segment']==s]['diversity'].mean())
    print(right_person_df[right_person_df['segment']==s]['right_person_name'].nunique())
    print(right_person_df[right_person_df['segment']==s]['diversity'].mean() * right_person_df[right_person_df['segment']==s]['right_person_name'].nunique())
    # print(right_person_df[right_person_df['segment']==s]['diversity'].mean() * right_person_df[right_person_df['segment']==s]['right_person_name'].nunique()/627)
    # print(right_person_df[right_person_df['segment']==s]['diversity'].mean())
ax.legend(loc='lower left', fontsize=18)
# ax.legend(loc='upper right', fontsize=18)

# ax.set_title('各期間における特許権者の補累積次数（Diversity）分布（両対数スケール）'+'\n', fontsize=20)
ax.set_xlabel('特許権者次数（Diversity）', fontsize=18)
ax.set_ylabel('ccdf', fontsize=18)

ax.set_xscale('log')
ax.set_yscale('log')

ax.tick_params(labelsize=18)
ax.set_xlim(0.8, 300)

# x軸の指数表記を普通に戻す魔法
ax.xaxis.set_major_formatter(ptick.ScalarFormatter(useMathText=True))

# ax.set_xlim(prop_dict['xlim'])
# ax.set_ylim(prop_dict['ylim'])

ax.grid(axis='both', 
        which='major', 
        alpha=1, 
        linestyle='--', 
        linewidth=0.6, 
        color='gray')
    
plt.show()


In [ ]:
# 特許権者同士のつながり
hr_G = bip.projected_graph(BG, hr_nodes)
hr_degree_dict = dict(hr_G.degree())

print(max(hr_degree_dict, key=hr_degree_dict.get))
# hr_degree_dict

In [ ]:
# 特許分類同士のつながり
ipc_G = bip.projected_graph(BG, ipc_nodes)
ipc_degree_dict = dict(ipc_G.degree())

print(max(ipc_degree_dict, key=ipc_degree_dict.get))
# ipc_degree_dict